In [1]:
# 1. Use beutifulsoup to 
# 2. Confirm loaded correctly
# 3. define def to find the table
# what does it return the table as? 

import pandas as pd
from bs4 import BeautifulSoup as bs

def getTables(htmlDoc):
	with open(htmlDoc) as fp:
		soup = bs(fp)
	return soup.findAll('table')
	
course_hist = getTables('course_history.html')

print(course_hist)

[<table width="800px"><tr><td colspan="2"><p>Undergraduate Credential Summary </p></td></tr><tr><td class="t1">Credential Awarded: </td><td>Bachelor of Arts</td></tr><tr><td class="t1">Plan: </td><td>Specialization Pure and Applied Mathematics</td></tr><tr><td class="t1">Confer Date: </td><td>05/10/2018</td></tr><tr><td class="t1">Final Graduation GPA: </td><td> 3.21</td></tr><tr><td colspan="2"></td></tr><tr><td colspan="2"><p>Undergraduate Academic Program History </p></td></tr><tr><td colspan="2"></td></tr><tr><td class="t1">Active in Program</td><td>26/08/2015</td></tr><tr><td class="t1">Admit Term </td><td>Fall 2015</td></tr><tr><td class="t1"> </td><td>Matriculated</td></tr><tr><td class="t1"> </td><td>Bachelor of Arts</td></tr><tr><td class="t1"> </td><td>Major Mathematics and Statistics</td></tr><tr><td colspan="2"></td></tr><tr><td class="t1">Active in Program</td><td>30/11/2015</td></tr><tr><td class="t1"> </td><td>Concentration Change</td></tr><tr><td class="t1"> </td><td>Ba

But all this does is find the table enclosed in the `table` tag. I want to get rid of all that and have what's in the cells. 

Try Pandas next

In [2]:
col_names = ['Course', 'Num','Section',	'Description',	
             'Attempted','Grade','Notation',	
             'GPA','Class Avg',	'Class Size',	
             'Credits Earned','Other']
len(col_names)

12

In [14]:
df_list = pd.read_html('course_history.html')

files = []
for i, df in enumerate(df_list):
#     print(df)
    df.to_csv('table_{}.csv'.format(i), header=None, index=False)
    files.append('table_{}.csv'.format(i))

In [56]:
dfs_list

['df_3', 'df_4', 'df_5', 'df_6', 'df_7', 'df_8']

In [67]:
cnt = len(files)  # your loop
dict_of_df = {} # initialize empty dictionary

for i in range(3,cnt):
    dict_of_df['df_{}'.format(i)] = pd.read_csv('table_{}.csv'.format(i), skiprows=1,usecols=[0,1,2,3], names=col_names[:4])

dfs_list = list(dict_of_df.keys())
df = pd.concat(dict_of_df, ignore_index=True)
# display(df)
df.drop(columns=['Section'], inplace=True)
display(df)

,Course,Num,Description
0,FRAN,305,COMMUNICATION ORALE
1,GEOL,210,INTRODUCTION TO THE EARTH
2,MAST,217,INTRO/MATHEMATICAL THINKING
3,MAST,219,MULTIVARIABLE CALCULUS II
4,MECH,211,MECHANICAL ENGR. DRAWING
5,GEOL,208,"THE EARTH, MOON AND PLANETS"
6,MATH,252,LINEAR ALGEBRA II
7,MATH,365,ANALYSIS II
8,STAT,249,PROBABILITY I
9,MATH,366,COMPLEX ANALYSIS I


In [101]:
df.info()

mask = df.Course == 'Course Topic:'
topics_ind = list(df[mask].index)
topics = list(df[mask].Num.values)

locations = list(zip(topics_ind, topics))
locations

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
Course         28 non-null object
Num            28 non-null object
Description    23 non-null object
dtypes: object(3)
memory usage: 752.0+ bytes


[(13, 'DISCRETE DYNAMICAL SYSTEMS'),
 (18, 'SET THEORY'),
 (23, 'ALGEBRAIC NUMBER THEORY'),
 (25, 'MATH 457 MCGILL'),
 (27, 'RANDOM MATRICES')]

In [112]:
for i in range(len(locations)):
    if i == 3:
        df.loc[locations[i][0]-1].Course = 'MATH'
        df.loc[locations[i][0]-1].Num = '457'
    else:
        df.iloc[locations[i][0]-1].Description = locations[i][1]

In [122]:
df.Num = df.Num.astype(str);
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 0 to 26
Data columns (total 3 columns):
Course         23 non-null object
Num            23 non-null object
Description    23 non-null object
dtypes: object(3)
memory usage: 736.0+ bytes


In [127]:
df['Course_Num'] = df['Course'] + ' ' + df['Num']
df.drop(columns=['Course', 'Num'], inplace=True)

In [130]:
df = df[['Course_Num', 'Description']]

In [140]:
df['Description'] = df['Description'].str.title()
df['Description']
df

,Course_Num,Description
0,FRAN 305,Communication Orale
1,GEOL 210,Introduction To The Earth
2,MAST 217,Intro/Mathematical Thinking
3,MAST 219,Multivariable Calculus Ii
4,MECH 211,Mechanical Engr. Drawing
5,GEOL 208,"The Earth, Moon And Planets"
6,MATH 252,Linear Algebra Ii
7,MATH 365,Analysis Ii
8,STAT 249,Probability I
9,MATH 366,Complex Analysis I


In [141]:
df.to_csv('course_history.csv')

In [142]:
df.to_html('course_history.html')